In [1]:
import pandas as pd

collections = pd.read_csv('item_collection_mapping.csv')
item_metadata = pd.read_csv('item_metadata.csv')

In [10]:
item_metadata['rarity'].value_counts()

rarity
Covert              681
Mil-Spec Grade      431
Restricted          300
Consumer Grade      200
Industrial Grade    192
Classified          176
Extraordinary        72
Contraband            1
Name: count, dtype: int64